# Demo for Unconditional Generation

In [7]:
# Designate GPU to use
import os
gpu_ids = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_ids}"

In [8]:
# import libraries
import numpy as np
from PIL import Image
from pathlib import Path
from IPython.display import Image as ipy_image
from IPython.display import display
from termcolor import colored, cprint

import torch
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import torchvision.utils as vutils
import torch.nn.functional as F

from datasets.dataloader import CreateDataLoader, get_data_generator

from collections import OrderedDict
from models.base_model import create_model

import joblib

from visualize.visualizer import MOFVisualizer, animate, resize_gif, make_concat_gif
from PIL import Image, ImageSequence
import matplotlib.pyplot as plt
import glob
import os

from utils.data_util import lattices_to_params_shape, get_gt_crys_ori

import warnings
warnings.filterwarnings("ignore")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from torch_geometric.data import Batch, Data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Options for the model. please check `utils/demo_util.py` for more details
from utils.demo_util import ChargeDIFFOpt

seed = 42

opt = ChargeDIFFOpt(gpu_ids=gpu_ids, seed=seed)

opt.init_dset_args()
opt.init_model_args()

device = opt.device

[*] CHGDIFF_TestOption initialized.


In [10]:
# Initialize ChargeDIFF model

opt.model='chargediff_uncond'
opt.vq_cfg="./configs/vqvae.yaml"
opt.vq_ckpt="./saved_ckpt/vqvae.pth"
opt.df_cfg = './configs/chargediff_uncond.yaml'
opt.ckpt = './saved_ckpt/july.pth'

CHARGEDIFF = create_model(opt)
cprint(f'[*] "{CHARGEDIFF.name()}" loaded.', 'cyan')

[*] VQVAE: weight successfully load from: ./saved_ckpt/vqvae.pth
[*] weight successfully load from: ./saved_ckpt/july.pth
[*] Model has been created: CHARGEDIFF-Model
[*] "CHARGEDIFF-Model" loaded.


In [13]:
from pymatgen.io.vasp.outputs import Chgcar, VolumetricData

batches = 20
batch_size = 50

for i in range(batches):

    gen_crys, chgden_tot = CHARGEDIFF.ab_initio_gen(num_batch = 1, batch_size = batch_size, dataset='mp_20_charge', cutoff = 0.5, step_lr = 1e-5)

    for j in range(len(gen_crys)):

        cry = gen_crys[j]
        chgden = chgden_tot[j]
        structure = cry.structure
        volumetric = VolumetricData(structure = structure, data={'total': chgden})

        idx = i*batch_size + j
        volumetric.write_file(f'./sample/real_uncond_1/vasp/{idx}.vasp')
        structure.to_file(f'./sample/real_uncond_1/cifs/{idx}.cif')
    

100%|██████████| 1000/1000 [02:44<00:00,  6.09it/s]


In [6]:
'''
from pymatgen.io.vasp.outputs import Chgcar, VolumetricData

def un_normalize(chgden, volume):
    max_val = 100.0
    min_val = 0.0001

    chgden = np.clip(chgden, 0.0, 1.0)
    
    return (10 ** (chgden * (np.log10(max_val) - np.log10(min_val)) + np.log10(min_val))) * volume


for i in range(len(gen_crys_1000)):

    cry = gen_crys_1000[i]
    chgden = chgden_tot_1000[i]
    
    #chgden  = un_normalize(chgden, 50)
    
    structure = cry.structure
    
    volumetric = VolumetricData(structure = structure, data={'total': chgden})

    volumetric.write_file(f'./sample/uncond_1000/{i}.vasp')
'''

"\nfrom pymatgen.io.vasp.outputs import Chgcar, VolumetricData\n\ndef un_normalize(chgden, volume):\n    max_val = 100.0\n    min_val = 0.0001\n\n    chgden = np.clip(chgden, 0.0, 1.0)\n    \n    return (10 ** (chgden * (np.log10(max_val) - np.log10(min_val)) + np.log10(min_val))) * volume\n\n\nfor i in range(len(gen_crys_1000)):\n\n    cry = gen_crys_1000[i]\n    chgden = chgden_tot_1000[i]\n    \n    #chgden  = un_normalize(chgden, 50)\n    \n    structure = cry.structure\n    \n    volumetric = VolumetricData(structure = structure, data={'total': chgden})\n\n    volumetric.write_file(f'./sample/uncond_1000/{i}.vasp')\n"